# Q1

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("data/results-gpt4o.csv")

In [3]:
df = df.iloc[:300]

In [4]:
df.head()

,answer_llm,answer_orig,document,question,course
0,The CONTEXT provided does not give specific in...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up for the Machine Learning Zoomc...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,"No, the provided context does not include any ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp


In [5]:
from sentence_transformers import SentenceTransformer

model_name = "multi-qa-mpnet-base-dot-v1"


/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
embedding_model = SentenceTransformer(model_name)

You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





In [7]:
answer_llm = df.iloc[0].answer_llm

In [8]:
emdedding = embedding_model.encode(answer_llm)

In [9]:
emdedding[0]

-0.23882452

# Q2

In [10]:
def compute_dot_product(embedding1, embedding2):
    return np.dot(embedding1, embedding2)

In [11]:
evaluations = []
for idx, row in df.iterrows():
    emb_llm = embedding_model.encode(row['answer_llm'])
    emb_orig = embedding_model.encode(row['answer_orig'])
    score = compute_dot_product(emb_llm, emb_orig)
    evaluations.append(score)

In [12]:
percentile_75 = np.percentile(evaluations, 75)

print("75th Percentile of Scores:", percentile_75)

75th Percentile of Scores: 32.09545612335205


# Q3

In [13]:
def normalize_vector(v):
    norm = np.linalg.norm(v)
    return v / norm

In [14]:
def compute_cosine_similarity(embedding1, embedding2):
    embedding1_norm = normalize_vector(embedding1)
    embedding2_norm = normalize_vector(embedding2)
    return np.dot(embedding1_norm, embedding2_norm)

In [15]:
evaluations = []
for idx, row in df.iterrows():
    emb_llm = embedding_model.encode(row['answer_llm'])
    emb_orig = embedding_model.encode(row['answer_orig'])
    score = compute_cosine_similarity(emb_llm, emb_orig)
    evaluations.append(score)


In [16]:
percentile_75 = np.percentile(evaluations, 75)

print("75th Percentile of Scores:", percentile_75)

75th Percentile of Scores: 0.8348871618509293


# Q4

In [17]:
! pip install rouge


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [19]:
from rouge import Rouge

In [20]:
r = df[df['document'] == '5170565b'].iloc[0]

In [22]:
rouge_scorer = Rouge()

scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]

print("ROUGE-1:", scores['rouge-1'])
print("ROUGE-2:", scores['rouge-2'])
print("ROUGE-L:", scores['rouge-l'])

ROUGE-1: {'r': 0.696969696969697, 'p': 0.5897435897435898, 'f': 0.6388888839236112}
ROUGE-2: {'r': 0.43243243243243246, 'p': 0.34782608695652173, 'f': 0.3855421637334882}
ROUGE-L: {'r': 0.6666666666666666, 'p': 0.5641025641025641, 'f': 0.6111111061458334}


# Q5

In [23]:
rouge_1_f1 = scores['rouge-1']['f']
rouge_2_f1 = scores['rouge-2']['f']
rouge_l_f1 = scores['rouge-l']['f']

average_rouge_f1 = (rouge_1_f1 + rouge_2_f1 + rouge_l_f1) / 3

print("ROUGE-1 F1:", rouge_1_f1)
print("ROUGE-2 F1:", rouge_2_f1)
print("ROUGE-L F1:", rouge_l_f1)
print("Average ROUGE F1:", average_rouge_f1)

ROUGE-1 F1: 0.6388888839236112
ROUGE-2 F1: 0.3855421637334882
ROUGE-L F1: 0.6111111061458334
Average ROUGE F1: 0.545180717934311


# Q6

In [24]:
rouge_scorer = Rouge()

rouge_1_scores = []
rouge_2_scores = []
rouge_l_scores = []
rouge_avg_scores = []

for idx, row in df.iterrows():
    scores = rouge_scorer.get_scores(row['answer_llm'], row['answer_orig'])[0]
    rouge_1 = scores['rouge-1']['f']
    rouge_2 = scores['rouge-2']['f']
    rouge_l = scores['rouge-l']['f']
    rouge_avg = (rouge_1 + rouge_2 + rouge_l) / 3
    
    rouge_1_scores.append(rouge_1)
    rouge_2_scores.append(rouge_2)
    rouge_l_scores.append(rouge_l)
    rouge_avg_scores.append(rouge_avg)

scores_df = pd.DataFrame({
    'rouge_1': rouge_1_scores,
    'rouge_2': rouge_2_scores,
    'rouge_l': rouge_l_scores,
    'rouge_avg': rouge_avg_scores
})

average_rouge_2 = scores_df['rouge_2'].mean()


print(scores_df)
print("Average ROUGE-2 Score:", average_rouge_2)

      rouge_1   rouge_2   rouge_l  rouge_avg
0    0.187500  0.052632  0.187500   0.142544
1    0.235294  0.103896  0.205882   0.181691
2    0.289474  0.091954  0.236842   0.206090
3    0.239130  0.035714  0.217391   0.164079
4    0.155440  0.028269  0.145078   0.109596
..        ...       ...       ...        ...
295  0.772277  0.697248  0.752475   0.740667
296  0.538462  0.413333  0.523077   0.491624
297  0.624000  0.542857  0.624000   0.596952
298  0.615385  0.517241  0.596154   0.576260
299  0.561983  0.453901  0.561983   0.525956

[300 rows x 4 columns]
Average ROUGE-2 Score: 0.2014210455831726
